# Learning Rate

Die Learning Rate (LR) legt fest, wie stark die Gewichte der Neuronen pro Trainingslauf angepasst werden, und beeinflusst somit direkt die Trainingsgeschwindigkeit des Netzwerkes. Die LR ist ein wichtiger Hyperparameter: Ist sie zu niedrig, kovergiert das Model nur sehr langsam. Ist sie zu hoch, kann die Fehlerfunktion unter Umständen überhaupt nicht minimiert werden. In diesem Notebook werden diese Fälle demonstriert, sowie der LR Finder ausprobiert.